In [ ]:
import os

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

transform = transforms.Compose([
    transforms.Resize((400, 400)),
    transforms.RandomCrop((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transforms = torch.nn.Sequential(
    transforms.CenterCrop(10),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
)

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print('Using {0} device'.format(device))

In [ ]:
class CountryDataset(Dataset):

    def __init__(self, root_dir, transform=None):

        self.samples = []
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):

        img = Image.open(os.path.join(self.root_dir)).convert('RGB')


        if self.transform is not None:
            img = self.transform(img)

        return (img, label)